In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader

import math
import random

import time

from tqdm import trange, tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class FFLayer(nn.Module):
    def __init__(self,features_out,features_in = None, bias = False):
        
        super().__init__()
        
        if features_in is None:
            features_in = features_out
        self.linear = nn.Linear(features_in,features_out,bias=bias)
        self.activation = nn.ReLU()
        
    def forward(self,x):
        x = self.linear(x)
        return self.activation(x)

class NeuralLayer(nn.Module):
    def __init__(self, features_in,features_out,num_layers=3):
        
        super().__init__()
        
        self.num_layers = num_layers
        self.linear1 = nn.Linear(features_in,features_out,bias=True)
        self.activation1 = nn.ReLU()
        
        self.ff_layers = nn.ModuleList(
            [FFLayer(features_out) for _ in range(num_layers)])
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        for i in range(self.num_layers):
            x = self.ff_layers[i](x)
        
        return x
    
class GraphLayer(nn.Module):
    def __init__(self, features_in,features_out,num_layers=3):
        
        super().__init__()
        
        self.num_layers = num_layers
        self.linear1 = nn.Linear(features_in,features_out,bias=True)
        self.activation1 = nn.ReLU()
        
        self.ff_layers = nn.ModuleList(
            [FFLayer(features_out) for _ in range(num_layers)])
        
    def forward(self, x, adj):
        x = torch.matmul(adj,x)
        x = self.linear1(x)
        x = self.activation1(x)
        for i in range(self.num_layers):
            x = self.ff_layers[i](x)
        
        return x
    
class GraphNetwork(nn.Module):
    def __init__(self, features_in, features_out,
                 features_nn, num_layers,
                 num_nn_layers, num_blocks):
        
        super().__init__()
        
        self.num_layers = num_layers
        self.num_blocks = num_blocks
        
        self.graphlayer1 = GraphLayer(features_in, features_out,
                                      num_layers)
        
        self.ln1 = nn.LayerNorm(features_out)
        
        self.graph_layers = nn.ModuleList(
            [GraphLayer(features_out,features_out,num_layers)\
             for _ in range(num_blocks-1)])
        
        self.norm_layers = nn.ModuleList([nn.LayerNorm(features_out) for _ in range(num_blocks -1)])
        
        self.ff1_u = FFLayer(features_out,bias = True)
        self.ff2_u = FFLayer(features_nn, features_out, bias = True)
        self.nn1_u = NeuralLayer(features_nn+2,features_nn+2,num_layers = num_nn_layers)
        
        self.linear_u = nn.Linear(features_nn+2, 1)
                
    def forward(self,x,adj,lab0,lab1):
        x= self.graphlayer1(x,adj)
        x = self.ln1(x)
        
        for i in range(self.num_blocks -1):
            res = x
            x = self.graph_layers[i](x,adj)
            x = torch.add(res,x)
            x = self.norm_layers[i](x)
        
        y = lab0
        z = lab1
        u = self.ff1_u(x)
        u = self.ff2_u(u)
        u,_ = torch.max(u,dim = 1)
        
        u = torch.cat([u,torch.add(y,z),torch.abs(torch.add(y,-z))],dim = -1)
        u = self.nn1_u(u)
        u = nn.Sigmoid()(self.linear_u(u))
            
        return u
    
    def count_params(self):
        return sum([param.nelement() for param in self.parameters()])

In [ ]:
class GraphDataset(Dataset):
    def __init__(self, df = None, path = None, use_cuda=False, pred = False):
        
        if path is not None:
            self.df = pd.read_pickle(path)
        
        else:
            self.df = df
        
    def __len__(self):
        return(len(self.df))
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        adj = torch.tensor(self.df.iloc[idx]['adj_matrix'])
        feat = torch.tensor(self.df.iloc[idx]['node_feature_matrix'])
        labl = torch.tensor(self.df.iloc[idx]['labels'])
        samp = {'adj':adj, 'feat':feat, 'labl':labl}
        
        return samp

In [ ]:
from scipy.linalg import fractional_matrix_power

def max_node(arr):
    arr = np.sum(arr,axis = 1)
    try:
        m = np.where(arr==0)[0].min()
    except ValueError:
        m = arr.shape[0]
    
    return m

def pad_out(arr,size):
    diff = size-arr.shape[0]
    return np.pad(arr,(0,diff),'constant',constant_values = 0)
    
def adj_norm(arr):
    size = arr.shape[0]
    m = max_node(arr)
    arr = arr[:m,:m]
    d = np.diag(np.sum(arr,axis=1))
    d_norm = fractional_matrix_power(d, -0.5)
    prod = np.matmul(d_norm,np.matmul(arr,d_norm))
    return pad_out(prod,size)

def direct_sum(arr1,arr2):
    dsum = np.zeros(np.add(arr1.shape,arr2.shape))
    dsum[:arr1.shape[0],:arr1.shape[1]]=arr1
    dsum[arr1.shape[0]:,arr1.shape[1]:]=arr2
    return dsum

def label_shuffle(df):
    data = df_combined[['adj_matrix','node_feature_matrix']]
    data['labels']= df['labels'].sample(frac = 1).reset_index(drop = True)
    return data

In [ ]:
data_path = 'data_path'

df = pd.read_pickle(data_path)
df['adj_matrix'] = df.apply(lambda x: direct_sum(x['adj_matrix_x'],x['adj_matrix_y']),axis = 1)
df['node_feature_matrix'] = df.apply(lambda x: np.concatenate((x['node_feature_matrix_x'], x['node_feature_matrix_y'])), axis = 1)


In [ ]:
test_dataset = GraphDataset(df, pred = False)

In [ ]:
use_cuda = True
device =  torch.device("cuda" if use_cuda else "cpu")
data_type = torch.float32

model = GraphNetwork(34,128,32,num_layers = 2,num_nn_layers = 10, num_blocks = 5)
model.to(device, dtype= data_type)

path = '../models/model_all_data_with_ac50_stand_v7'
model.load_state_dict(torch.load(path))

test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=0, sampler = None)

In [ ]:
def pred(model, test_dataloader, use_cuda = False):
    if use_cuda is None:
        use_cuda = torch.cuda.is_available()
        
    device =  torch.device("cuda" if use_cuda else "cpu")
    
    data_type = torch.float32
    
#     model.to(device, dtype= data_type)
    model.eval()
    
    preds = []
#     true = []
    for samp in test_dataloader:
        samp['feat']=samp['feat'].to(device,dtype=data_type)
        samp['adj']=samp['adj'].to(device,dtype=data_type)
        samp['labl']=samp['labl'].to(device,dtype=data_type)
        
        preds.append(torch.cat(list([model(samp['feat'],samp['adj'],samp['labl'][:,[0]],samp['labl'][:,[1]])]),dim = 1))
#         true.append(samp['labl'][:,[3]])
        
    return torch.cat(preds).detach()#,torch.cat(true)

In [ ]:
y_score = pred(model, test_dataloader, use_cuda = True)